# Setup

In [6]:
# Weights and Biases
#!pip install -q wandb
# Tensorflow
#!pip install -q tensorflow

In [1]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tensorflow import square, reduce_mean
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import Callback
from tensorflow.math import multiply
from math import log

2023-02-16 15:56:53.975659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# If running in colab, insert your wandb key here

import config
#Erlend
#wandb.login(key=config.erlend_key)
# Hjalmar
wandb.login(key=config.hjalmar_key)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vinje (avogadro). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/hjalmarjacobvinje/.netrc


True

# Load, split and normalize data

### Load data

In [6]:
google_colab = False

if google_colab:
    import tensorflow as tf
    # Pring info
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)
    
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

    # Code to read csv file into Colaboratory:
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    id = "1tSZeYgiZh_gL6CKUYufNlb1C6X4vODY3"
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('2021_2022.csv')  
    df_read = pd.read_csv('2021_2022.csv')
else:
    file = "../data/processed_data/2021_2022.csv"
    df_read = pd.read_csv(file)

display(df_read)

,Unnamed: 0,Quote_date,Expire_date,Price,Underlying_last,Strike,TTM,Moneyness,Underlying_return,Underlying_1,...,Underlying_142,Underlying_143,Underlying_144,Underlying_145,Underlying_146,Underlying_147,Underlying_148,Underlying_149,Underlying_150,R
0,0,2021-01-04,2021-01-06,2701.855,3701.38,1000.0,2,3.701380,-0.014623,0.006403,...,0.012285,0.026023,-0.003299,0.013479,0.008299,0.003573,0.005010,-0.002062,0.014591,0.09
1,1,2021-01-04,2021-01-06,2598.795,3701.38,1100.0,2,3.364891,-0.014623,0.006403,...,0.012285,0.026023,-0.003299,0.013479,0.008299,0.003573,0.005010,-0.002062,0.014591,0.09
2,2,2021-01-04,2021-01-06,2500.195,3701.38,1200.0,2,3.084483,-0.014623,0.006403,...,0.012285,0.026023,-0.003299,0.013479,0.008299,0.003573,0.005010,-0.002062,0.014591,0.09
3,3,2021-01-04,2021-01-06,2400.290,3701.38,1300.0,2,2.847215,-0.014623,0.006403,...,0.012285,0.026023,-0.003299,0.013479,0.008299,0.003573,0.005010,-0.002062,0.014591,0.09
4,4,2021-01-04,2021-01-06,2300.300,3701.38,1400.0,2,2.643843,-0.014623,0.006403,...,0.012285,0.026023,-0.003299,0.013479,0.008299,0.003573,0.005010,-0.002062,0.014591,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4058478,4058478,2022-12-30,2025-12-19,6.950,3839.81,8400.0,1085,0.457120,-0.002300,0.017343,...,-0.029149,-0.023985,-0.010617,0.009520,0.003184,-0.016515,0.018488,-0.007915,-0.005897,4.22
4058479,4058479,2022-12-30,2025-12-19,5.950,3839.81,8600.0,1085,0.446490,-0.002300,0.017343,...,-0.029149,-0.023985,-0.010617,0.009520,0.003184,-0.016515,0.018488,-0.007915,-0.005897,4.22
4058480,4058480,2022-12-30,2025-12-19,5.450,3839.81,8800.0,1085,0.436342,-0.002300,0.017343,...,-0.029149,-0.023985,-0.010617,0.009520,0.003184,-0.016515,0.018488,-0.007915,-0.005897,4.22
4058481,4058481,2022-12-30,2025-12-19,4.500,3839.81,9000.0,1085,0.426646,-0.002300,0.017343,...,-0.029149,-0.023985,-0.010617,0.009520,0.003184,-0.016515,0.018488,-0.007915,-0.005897,4.22


### Format input data

In [16]:
df = df_read
del df_read

# Format settings
max_timesteps = 90
moneyness = False # Moneyness = True WIP
bs_vars = ['Moneyness', 'TTM', 'R'] if moneyness else ['Underlying_last', 'Strike', 'TTM', 'R']
underlying_lags = ['Underlying_last'] + [f'Underlying_{i}' for i in range (1, max_timesteps)]
strikes = ["Strike"] + [f'Strike_{i}' for i in range (1, max_timesteps)]


# Create train, validation and test set split points
train_start = datetime(2021,1,1) 
val_start = train_start + relativedelta(months=1) 
test_start = val_start + relativedelta(months=1)
test_end = test_start + relativedelta(months=1)
train_start = str(train_start.date())
val_start = str(val_start.date())
test_start = str(test_start.date())
test_end = str(test_end.date())

# Split train and validation data
df_train = df[(df['Quote_date'] >= train_start) & ( df['Quote_date'] < val_start)]
df_val = df[(df['Quote_date'] >= val_start) & ( df['Quote_date'] < test_start)]

# Add timestep number of duplicates of the strike price
for i in range(1, max_timesteps):
    df_train[f'Strike_{i}'] = df_train['Strike']
    df_val[f'Strike_{i}'] = df_val['Strike']

display(df_train.head(5))

del df
# Extract target values
train_y = (df_train['Price']/df_train['Strike']).to_numpy() if moneyness else df_train['Price'].to_numpy()
val_y = (df_val['Price']/df_val['Strike']).to_numpy() if moneyness else df_val['Price'].to_numpy()

# If usining moneyness, extract strike
if moneyness:
    train_strike = df_train['Strike'].to_numpy()
    val_strike = df_val['Strike'].to_numpy()

# Convert dataframes to numpy arrays
# Make train_x an array for the LSTM network with shape (samples, timesteps, features)
train_x = [df_train[underlying_lags + strikes].to_numpy().reshape(-1, max_timesteps, 2), df_train[bs_vars].to_numpy()]
val_x = [df_val[underlying_lags + strikes].to_numpy().reshape(-1, max_timesteps, 2), df_val[bs_vars].to_numpy()]

del df_train
del df_val

print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')

# Scale features based on training set
underlying_scaler = MinMaxScaler()
train_x[0] = underlying_scaler.fit_transform(train_x[0].reshape(-1, train_x[0].shape[-1])).reshape(train_x[0].shape)
val_x[0] = underlying_scaler.transform(val_x[0].reshape(-1, val_x[0].shape[-1])).reshape(val_x[0].shape)

bs_scaler = MinMaxScaler()
train_x[1] = bs_scaler.fit_transform(train_x[1])
val_x[1] = bs_scaler.transform(val_x[1])

# Shuffle training set
np.random.seed(0)
shuffle = np.random.permutation(len(train_x[0]))
train_x = [train_x[0][shuffle], train_x[1][shuffle]]
train_y = train_y[shuffle]
if moneyness:
    train_strike = train_strike[shuffle]

# Reshape data to fit LSTM
train_x = [train_x[0].reshape(len(train_x[0]), max_timesteps,2), train_x[1]]
val_x = [val_x[0].reshape(len(val_x[0]), max_timesteps, 2), val_x[1]]

print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')

/var/folders/wk/x86_p6511l95p594k6qnb98h0000gn/T/ipykernel_72433/3138044945.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train[f'Strike_{i}'] = df_train['Strike']
/var/folders/wk/x86_p6511l95p594k6qnb98h0000gn/T/ipykernel_72433/3138044945.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val[f'Strike_{i}'] = df_val['Strike']
/var/folders/wk/x86_p6511l95p594k6qnb98h0000gn/T/ipykernel_72433/3138044945.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,Unnamed: 0,Quote_date,Expire_date,Price,Underlying_last,Strike,TTM,Moneyness,Underlying_return,Underlying_1,...,Strike_80,Strike_81,Strike_82,Strike_83,Strike_84,Strike_85,Strike_86,Strike_87,Strike_88,Strike_89
0,0,2021-01-04,2021-01-06,2701.855,3701.38,1000.0,2,3.701380,-0.014623,0.006403,...,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0,1000.0
1,1,2021-01-04,2021-01-06,2598.795,3701.38,1100.0,2,3.364891,-0.014623,0.006403,...,1100.0,1100.0,1100.0,1100.0,1100.0,1100.0,1100.0,1100.0,1100.0,1100.0
2,2,2021-01-04,2021-01-06,2500.195,3701.38,1200.0,2,3.084483,-0.014623,0.006403,...,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0
3,3,2021-01-04,2021-01-06,2400.290,3701.38,1300.0,2,2.847215,-0.014623,0.006403,...,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0,1300.0
4,4,2021-01-04,2021-01-06,2300.300,3701.38,1400.0,2,2.643843,-0.014623,0.006403,...,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0,1400.0


Train shape: (140302, 90, 2), (140302, 4)
Val shape: (135252, 90, 2), (135252, 4)
Train shape: (140302, 90, 2), (140302, 4)
Val shape: (135252, 90, 2), (135252, 4)


# Model construction

In [23]:
def create_model(config):
    '''Builds an LSTM-MLP model of minimum 2 layers sequentially from a given config dictionary'''

    # Input layers
    underlying_history = Input((config.LSTM_timesteps,2))
    bs_vars = Input((config.Num_features,))

    # LSTM layers
    model = Sequential()

    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        input_shape = (config.LSTM_timesteps, 2),
        return_sequences = True
    ))

    for _ in range(config.LSTM_layers - 2):
        model.add(LSTM(
            units = config.LSTM_units,
            activation = tanh,
            return_sequences = True
        ))
    
    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        return_sequences = False
    ))

    # MLP layers
    layers = Concatenate()([model(underlying_history), bs_vars])
    
    for _ in range(config.MLP_layers - 1):
        layers = Dense(config.MLP_units)(layers)
        layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
        layers = LeakyReLU()(layers)

    output = Dense(1, activation='relu')(layers)

    # Exponential decaying learning rate
    lr_schedule = ExponentialDecay(
        initial_learning_rate = config.Lr,
        decay_steps = int(len(train_x[0])/config.Minibatch_size),
        decay_rate=config.Lr_decay
    )

    # Compile model
    model = Model(inputs=[underlying_history, bs_vars], outputs=output)
    model.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

    model.summary()
    return model

# Hyperparameter search setup

In [18]:
# Configuring the sweep hyperparameter search space
sweep_configuration = {
    'method': 'random',
    'name': 'LSTM-MLP v.2.1 testing : TTM in LSTM',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        'LSTM_units': {
            'values': [4, 8, 16, 32]},
        'MLP_units': {
            'values': [50, 100, 200, 400, 600]},
        'LSTM_timesteps': {
            'values': [10, 20, 40, 60, 90]},
        'LSTM_layers': {
            'distribution': 'int_uniform',
            'max': 8, 'min': 2},
        'MLP_layers': {
            'distribution': 'int_uniform',
            'max': 8, 'min': 2},
        'Bn_momentum': {
            'values': [0.1, 0.4, 0.7, 0.99]},
        'Lr': {
            'distribution': 'log_uniform',
            'max': log(0.1), 'min': log(0.0001)},
        'Lr_decay': {
            'distribution': 'log_uniform',
            'max': log(1), 'min': log(0.8)},        
        'Minibatch_size': {
            'value': 4096},
        'Min_delta': {
            'value': 0.01 if moneyness else 1},
        'Patience': {
            'value': 20},
        'Num_features': {
            'value': 3 if moneyness else 4},
    }
}

# Initialize sweep and creating sweepID

# If new sweep, uncomment the line below and comment the line after it
sweep_id = wandb.sweep(sweep=sweep_configuration, project='Deep learning for option pricing - test area') 
#sweep_id = '98bxt6oq'

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.
wandb: WARNING   Violation 2. Lr_decay uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: 93294dmp
Sweep URL: https://wandb.ai/avogadro/Deep%20learning%20for%20option%20pricing%20-%20test%20area/sweeps/93294dmp


# Run hyperparameter search

In [19]:
#WIP
class MSE_LossCallback(Callback):
    def __init__(self, train_x, train_y, train_strike, val_x, val_y, val_strike):
        self.train_x = train_x
        self.train_y = train_y
        self.train_strike = train_strike
        self.val_x = val_x
        self.val_y = val_y
        self.val_strike = val_strike
    
    def on_epoch_end(self, epoch, logs={}):
        train_pred = self.model(train_x)
        val_pred = self.model(val_x)

        train_mse = reduce_mean(square(multiply(train_pred[:,0] - self.train_y, self.train_strike)))
        val_mse = reduce_mean(square(multiply(val_pred[:,0] - self.val_y, self.val_strike)))

        print(f' Training scaled MSE: {train_mse}, Validation scaled MSE: {val_mse}')


In [20]:
# Calculate the training and validation MSE loss on the actual option price when using price/strike as the target
def MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike):
    train_pred = model(train_x)
    val_pred = model(val_x)

    train_mse = reduce_mean(square((train_pred[:,0] - train_y)*train_strike))
    val_mse = reduce_mean(square((val_pred[:,0] - val_y)*val_strike))

    print(f' Training scaled MSE: {train_mse}, Validation scaled MSE: {val_mse}')

In [24]:
def trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = None, project = None):
    # Initialize a new wandb run
    with wandb.init(config=config, project = project):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # Build model and create callbacks
        model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta = config.Min_delta,
            patience = config.Patience,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )

        if moneyness:
            mse_callback = MSE_LossCallback(train_x, train_y, train_strike, val_x, val_y, val_strike)

        # Adapt sequence length to config
        train_x_adjusted = [train_x[0][:, :config.LSTM_timesteps, :], train_x[1]]
        val_x_adjusted = [val_x[0][:, :config.LSTM_timesteps, :], val_x[1]]
        print(f'Train shape: {train_x_adjusted[0].shape}, {train_x_adjusted[0].shape}')
        print(f'Val shape: {val_x_adjusted[0].shape}, {val_x_adjusted[0].shape}')

        # Train model
        model.fit(
            train_x_adjusted,
            train_y,
            batch_size = config.Minibatch_size,
            validation_data = (val_x_adjusted, val_y),
            epochs = 3,
            callbacks = [early_stopping, wandb_callback, mse_callback] if moneyness else [early_stopping, wandb_callback],
        )

        if moneyness:
            MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike)

### Run full sweep

In [25]:
wandb.agent(sweep_id=sweep_id, function=trainer, project='Deep learning for option pricing - test area', count = 3)

wandb: Agent Starting Run: r2duvyr8 with config:
wandb: 	Bn_momentum: 0.1
wandb: 	LSTM_layers: 2
wandb: 	LSTM_timesteps: 90
wandb: 	LSTM_units: 4
wandb: 	Lr: 0.005166651378034535
wandb: 	Lr_decay: 0.8762360318729152
wandb: 	MLP_layers: 8
wandb: 	MLP_units: 50
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 4096
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 90, 2)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 4)            256         ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 8)            0           ['sequential[0][0]',             
                                                                  'input_2[0][0]']            

epoch,▁▅█
loss,█▅▁
val_loss,█▅▁
best_epoch,2
best_val_loss,899855.375
epoch,2
loss,853225.25
val_loss,899855.375


wandb: Agent Starting Run: dtx1cqss with config:
wandb: 	Bn_momentum: 0.4
wandb: 	LSTM_layers: 2
wandb: 	LSTM_timesteps: 90
wandb: 	LSTM_units: 16
wandb: 	Lr: 0.000569366304717662
wandb: 	Lr_decay: 0.991813435624823
wandb: 	MLP_layers: 8
wandb: 	MLP_units: 200
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 4096
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 90, 2)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 16)           3328        ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 20)           0           ['sequential[0][0]',             
                                                                  'input_2[0][0]']            

epoch,▁▅█
loss,█▄▁
val_loss,█▅▁
best_epoch,2
best_val_loss,928028.875
epoch,2
loss,874433.4375
val_loss,928028.875


wandb: Agent Starting Run: ff6gai3r with config:
wandb: 	Bn_momentum: 0.7
wandb: 	LSTM_layers: 3
wandb: 	LSTM_timesteps: 90
wandb: 	LSTM_units: 8
wandb: 	Lr: 0.06196715999583398
wandb: 	Lr_decay: 0.8575104586169809
wandb: 	MLP_layers: 5
wandb: 	MLP_units: 100
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 4096
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 90, 2)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 8)            1440        ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 12)           0           ['sequential[0][0]',             
                                                                  'input_2[0][0]']            

epoch,▁▅█
loss,█▁▁
val_loss,█▄▁
best_epoch,2
best_val_loss,330.64856
epoch,2
loss,964.31653
val_loss,330.64856


### Single run

In [ ]:
config = {
    'LSTM_units': 8,
    'MLP_units': 100,
    'LSTM_timesteps': 20,
    'LSTM_layers': 3,
    'MLP_layers': 4,
    'Bn_momentum': 0.99,
    'Lr': 0.01,
    'Lr_decay': 1,
    'Minibatch_size': 4096,
    'Min_delta': 0.01 if moneyness else 1,
    'Patience': 20,
    'Num_features': 3 if moneyness else 4, 
    'Architecture': 'LSTM-MLP v.0.1',
    'Dataset': f'Moneyness. Train_start: {train_start}, val_start: {val_start}, test_start: {test_start}',
}
trainer(config = config, project = 'Deep learning for option pricing - test area')